In [3]:
!pip install pyspark


In [7]:
!spark-submit --version


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.28
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.


In [10]:
from pyspark.sql import SparkSession

In [18]:
spark_session=SparkSession.builder.appName('test').getOrCreate()

In [23]:
print(spark_session.conf.get('spark.app.id'))
print(spark_session.conf.get('spark.app.name'))

local-1755428994284
test


In [29]:
# spark_session.sparkContext.getConf().getAll()

In [33]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, regexp_extract

data = [(1, 'Winston', 'winston@leetcode.COM')]

schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("mail", StringType(), True)
])

users_df = spark_session.createDataFrame(data, schema)

# Filter for emails containing '@leetcode.com' and matching the specified prefix pattern
users_df = users_df.filter(col("mail").rlike("^[a-zA-Z][a-zA-Z0-9_.-]*@leetcode\\.com$"))

users_df.show(truncate=False)

+-------+---------+-----------------------+
|user_id|name     |mail                   |
+-------+---------+-----------------------+
|1      |Winston  |winston@leetcode.com   |
|3      |Annabelle|bella-@leetcode.com    |
|4      |Sally    |sally.come@leetcode.com|
+-------+---------+-----------------------+



In [37]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col

data = [(1, 'Abbot'),
        (2, 'Doris'),
        (3, 'Emerson'),
        (4, 'Green'),
        (5, 'Jeames')]

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("student", StringType(), True)
])

students_df = spark_session.createDataFrame(data, schema)
students_df=students_df.s

In [ ]:
from pyspark.sql.functions import when, col, max

# Get the maximum id from the students_df
max_id_df = students_df.agg(max("id").alias("max_id"))
max_id = max_id_df.collect()[0]["max_id"]

# Apply the logic to swap ids
reordered_students_df = students_df.withColumn(
    "id",
    when((col("id") % 2 == 1) & (col("id") + 1 <= max_id), col("id") + 1)
    .when(col("id") % 2 == 0, col("id") - 1)
    .otherwise(col("id"))
)

# Order the result by the new id
reordered_students_df = reordered_students_df.orderBy("id")

reordered_students_df.show()